In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import csv
from dateutil import parser

pd.set_option("display.max_colwidth", None)

c:\Users\tomme\AppData\Local\Programs\Python\Python311\Lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
def crawl_by_keyword(keyword):
    df=pd.read_csv('thanhnien_'+keyword+'.csv')
    keyword_lookup = keyword
    csv_file = open('thanhnien_'+keyword+'_extracted.csv', 'w', newline='',encoding='utf-8')
    writer = csv.writer(csv_file)
    writer.writerow(['Title', 'URL','Publish Date', 'Mentioned Times','Word Count'])

    for index, row in df.iloc[0:].iterrows():
        title = row['Title']
        url = row['URL']
        news_url = 'https://thanhnien.vn'+url
        response = requests.get(news_url)
        if response.status_code == 403:
            break
        write_flag = True

        soup=BeautifulSoup(response.content,'html.parser')
        try: 
            publish_date = soup.find('div',{'data-role': 'publishdate'}).get_text()
            publish_date=publish_date.replace('\n', ' ').replace('\t', ' ')
            cleaned_publish_date = publish_date.strip()
        except AttributeError:
            print('Could not get date of publication of: \n' + str(news_url))
            write_flag =False

        try:
            news_content = soup.find('div',class_='detail__main').get_text()
            nofm = news_content.lower().count(keyword_lookup.lower())
            nofw = len(news_content.lower().split())
        except  AttributeError:
            print('Cant target news content of: \n' + str(news_url))
            try:
                
                news_content = soup.find('div',class_='detail__cmain').get_text()
                nofm = news_content.lower().count(keyword_lookup.lower())
                nofw = len(news_content.lower().split())
                print('Target cmain instead successfully \n')
            except:
                print('Cant target news content cmain of: \n' + str(news_url) + '\n')
                write_flag = False

        try:
            # Parse the cleaned time string using dateutil.parser
            parsed_time = parser.parse(cleaned_publish_date)
            
            # Extract the date component
            date_only = parsed_time.date()
        except ValueError:
            print("Invalid time format. of: \n" + str(news_url))
        if write_flag == True:
            writer.writerow([title, url,date_only,nofm,nofw])
    csv_file.close()

In [3]:
keywords = [
    "công nghệ số"
]
for text in keywords:
    crawl_by_keyword(text)

Cant target news content of: 
https://thanhnien.vn/bo-truong-le-minh-hoan-dung-trach-the-he-tre-khong-chon-hoc-nong-nghiep-185230804200340177.htm
Target cmain instead successfully 

Cant target news content of: 
https://thanhnien.vn/truong-dh-kinh-te-tai-chinh-tphcm-ky-ket-hop-tac-voi-16-doanh-nghiep-185230706105128139.htm
Target cmain instead successfully 

Could not get date of publication of: 
https://thanhnien.vn/chung-ta-phai-dinh-hinh-mot-co-hoi-vang-rat-khac-cho-viet-nam-18523061022075295.htm
Cant target news content of: 
https://thanhnien.vn/chung-ta-phai-dinh-hinh-mot-co-hoi-vang-rat-khac-cho-viet-nam-18523061022075295.htm
Target cmain instead successfully 

Cant target news content of: 
https://thanhnien.vn/thu-tuong-se-doi-thoai-voi-thanh-nien-nguoi-tre-kien-nghi-dieu-gi-185230321212717753.htm
Target cmain instead successfully 

Cant target news content of: 
https://thanhnien.vn/giam-doc-dhqg-tphcm-khong-nen-bien-nha-khoa-hoc-thanh-ke-toan-vien-1851535333.htm
Target cmain in